# Validation - BAC response of models

This notebook evaluates qualitatively the capability of fitted models to reproduce BAC behavior.

In [ ]:
import json
import sys
import os
import time
import neuron
from pathlib import Path

import matplotlib.pyplot as plt
import neuroplotlib as nplt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import LFPy
import neuron
import numpy as np

import shutil

import model
import evaluator
import plotting
import utils

# %matplotlib notebook
%matplotlib notebook

In [ ]:
probe_type = "planar"
model_name = "hay" #'hallermann' # "hay" hay_ais
model_folder = (Path(".") / f"{model_name}_model").absolute()

In [ ]:
# compile model-specific mechanisms
if not (model_folder / "x86_64").is_dir():
    curr_dir = Path(".").absolute()
    os.chdir(model_folder)
    print(os.getcwd())
    os.system("nrnivmodl mechanisms")
    os.chdir(curr_dir)
else:
    print(f"Mechanisms already compiled for {model_name}")
    compiled_folder = Path("./x86_64")
    if compiled_folder.is_dir():
        shutil.rmtree(compiled_folder)
    shutil.copytree(model_folder / "x86_64", compiled_folder)

In [ ]:
protocols_file = "../data/hay_ecode_probe_planar/efeatures/protocols_BPO.json"
features_file = "../data/hay_ecode_probe_planar/efeatures/features_BPO.json"

In [ ]:
ais_recordings = {}

In [ ]:
cell = model.create(model_name=model_name, release=True, v_init=-65)

probe = model.define_electrode(probe_type=probe_type)
# probe=None

param_names = [param.name for param in cell.params.values() if not param.frozen]

if model_name == 'hallermann':
    sim = ephys.simulators.LFPySimulator(cell, cvode_active=False, electrode=probe)
else:
    sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe)

# Test calcium spike

In [ ]:
cell.freeze(param_dict={})

In [ ]:
cell.instantiate(sim=sim)

In [ ]:
lfpy_cell = cell.LFPyCell

## TODO: use neuron instead of LFPy and apply the exact same stimuli as the Hay paper

In [ ]:
soma_args = {
         'idx': 0,
         'record_current': True,
         'pptype': 'IClamp',
         'amp': 1,
         'dur': 5,
         'delay': 10
}

stim = LFPy.StimIntElectrode(cell=lfpy_cell, **soma_args)

synapseParameters = {
     'idx': lfpy_cell.get_closest_idx(x=0, y=620, z=0),
     'e': 0,                               # reversal potential
     'syntype': 'ExpSyn',                  # synapse type
     'tau': 2,                             # syn. time constant
     'weight': 1,                          # syn. weight
     'record_current': True                # syn. current record
}

synapse = LFPy.Synapse(lfpy_cell, **synapseParameters)
synapse.set_spike_times(np.array([13]))

In [ ]:
lfpy_cell.simulate(rec_vmem=True)

In [ ]:
plt.figure()
d1 = lfpy_cell.get_closest_idx(x=0, y=400, z=0)
d2 = lfpy_cell.get_closest_idx(x=0, y=620, z=0)
plt.plot(lfpy_cell.tvec, lfpy_cell.vmem[[0, d1, d2]].T)

In [ ]:
cell.destroy

In [ ]:
syn = lfpy_cell.synapses[0]
plt.figure()
plt.plot(lfpy_cell.tvec, syn.i)

In [ ]:
cell.unfreeze({})

## TODO: load optimized parameters and visualize BAC responses 